# Assignment: Ensemble Methods: DSEI210-Fall-2019: Michael Grossberg

In this assignment we are going to get comfortable combining and leveraging ML algorithms with ensemble methods. In order to get respectable results we would more certainly need to spend much more time on data normalization and cleaning than we will be able to. Also we are having you subsample the data severly (so it will run in a reasonable amount of time) which will have a strong negative effect on classification.

**Background:** Suppose you are given a collection of high-importance dermatoscopic images of skin lesions. With a high degree of certainty, you want to determine whether or not a particular lesion is malignant or benign. 

**Objective:** Experiment with different ensemble paradigms combined with different classifiers. See if ensembles help and how. How do they effect accruacy, precision, recall, f1 and variance vs. bias.

------------------------------------------------------------------------

## Introductory Data Analysis and Preprocessing

The path here will be to go from data and meta data to a standard matrix 'X' where the rows are observations, and the columns are features, in this case pixels. We also have a 'y' which is a boolean '0' for a benign kind of cancer and '1' for a malignant kind.

### My Imports

Here are the extensive list of imports for the whole notbook I had. You may have some differences in your solution. There are other ways to resize and import images. If you prefer those you are free to use them but you should, at least, know how to use these libraries. It is a very good idea to keep constants that are frequently used in your data at the TOP of the file. This is starndard across software development. In particular don't hide data paths deep in your code. When you get the repo there will be a zip file for the data. Unzip that file wherever you want (usually outside your your repo) but use the path variable to point to that location (here called `DATA_PATH`). You will also
come up with a location for `temp` data we called `PROCESSED_DATA_PATH`. This is where you save out your preprocessed images and the data for your `X` and `y`. This is important because you may find you have to kill your kernel, restart some other time and you want to save your work so you can resume.

<div></div>
<hr/>
<div style="background-color:azure;color:red"> <em>Tip:</em> Some of the algorithms run really slow. For debugging purposes only, run the algorithms on 1/4 or 1/8 of the data. Then double it and measure the running times. Before you run the bagging algorithms you should have some idea how long it will take to run the whole algorithm. Keep that in mind when you change parameters and explore better performance. Also I have included an import for FloatProgress. Learn how to use that, particularly in the preprocessing, so you know how far you are at any time.</div>

In [1]:
# Standard Stuff and Pre-processing
import numpy as np
import pandas as pd
import os
import os.path as op
import json
from skimage.io import imread, imsave
from skimage.transform import resize
from ipywidgets import FloatProgress
from matplotlib import pyplot as plt
%matplotlib inline

# Fix imbalanced data sets
from imblearn.over_sampling import SMOTE

#Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# This is the relative path where the data is stored
#DATA_PATH = "C:/Users/sonali/Desktop/DS/ml/ensemble-assignment/SampledImages/SampledImages"
#PROCESSED_DATA_PATH = "C:/Users/sonali/Desktop/DS/ml/ensemble-assignment/SampledImages/SampledImages/processed_images"
#PROCESSED_DATA_PATH = "C:/Users/sonali/Desktop/DS/ml/ensemble-assignment/SampledImages/SampledImages/data/

In [2]:
DATA_PATH="C:/Users/sonali/Desktop/DS/ml/ensemble-assignment/SampledImages"
PROCESSED_DATA_PATH="C:/Users/sonali/Desktop/DS/ml/ensemble-assignment/processed_images"
not_df_cached = not op.exists(PROCESSED_DATA_PATH)
print(not_df_cached)

False


## Unzip the dataset

Unzip the data set from the repo or download it. Put the SampledImages directory in some location and change the data path accordingly. You will see two kinds of files. One kind of file end with ".jpeg" and the other have no extension. 

* Flip through the images manually
* We will create a directory to hold processed images which we will rite to disk and stored in `PROCESSED_DATA_PATH`.
* Use the os and the os.path python library to check if the directory exists and if not create it
* You can loop through the `DATA_PATH` by reading the contents of the directory with `os.listdir` and running a for loop. We will treat the `jpeg` files and the metadata files separately. Keep in mind that we will build our `X` matrix of classification input from our images and our `y` classification outputs from our metadata.
* Lets focus on the metadata first. This meta data is in JSON form so you can parse them using the python json library imported above. You can either read them in as text and then use the json library to parse and convert it to a dictionary, or read it directly in as a JSON file. For each file we will look for a sub-key with name `benign_malignant`. If that key is present and the key has the value `malignant` the corresponding target value in `y` should be 1. A small number of rows are missing this key. In that case check the `diagnosis` key. In this case if the `diagnosis` is `basal cell carcinoma` then `y` should also be 1 otherwise, for other `diagnosis` when there is no `benign_malignant` key. Finally if there is no `benign_malignant` and no `diagnosis` value, the meta-data and corresponding image, must be ignored.
* In order to make the problem tractable you will read in an image using imread, resize the images to 25x25 pixels and store the image as a row in a matrix which will have one row for each image and 25x25x3 columns with 3 being R-G-B color channels. It is also worthwile to save out the images using imsave with a slightly different file name (like tacking on `small` before the `.jpeg`) in your `PROCESSED_DATA_PATH` directory. When you have created a matrix `X` with one row for each image for which we have a meta-data value 0 or 1 in `y`, save both `X` and `y` in the numpy file format (.npy) usin the command `np.save`. Keep in mind that we should check that this file(s) exists first before processing the images each time you run the notebook. This way you can start and stop your analysis loading from the file, if you processed the data or recomputing if you don't have them proceessed. You can manaually override the test when you need to so it will compute everyting again.


In [4]:
data=os.listdir(DATA_PATH)
data[0:5]


['ISIC_0000004',
 'ISIC_0000004.jpeg',
 'ISIC_0000016',
 'ISIC_0000016.jpeg',
 'ISIC_0000049']

In [5]:
#seprate meta data from data
meta_data=[i for i in data if len(i)==12]
metadata_file=sorted(meta_data)
metadata_file[0:5]

['ISIC_0000004',
 'ISIC_0000016',
 'ISIC_0000049',
 'ISIC_0000059',
 'ISIC_0000063']

In [10]:
#seprate images file from data
images=[i for i in data if len(i)==17]
image_file=sorted(images)
image_file[0:5]


['ISIC_0000004.jpeg',
 'ISIC_0000016.jpeg',
 'ISIC_0000049.jpeg',
 'ISIC_0000059.jpeg',
 'ISIC_0000063.jpeg']

In [3]:
data_file = list(map(lambda x: DATA_PATH + '/' + x,sorted(os.listdir(DATA_PATH))))


In [4]:
if not os.path.exists(PROCESSED_DATA_PATH):
        os.makedirs(PROCESSED_DATA_PATH)

In [5]:
def images():
    X = []
    y = []

    #data_file
    for i in range(0, len(data_file), 2):
        with open(data_file[i],'r') as f:
            file = json.loads(f)
            
            
        if 'benign_malignant' in file['meta']['clinical'] and file['meta']['clinical']['benign_malignant']:
            if file['meta']['clinical']['benign_malignant'] == 'malignant':
                y.append(1)
            else:
                y.append(0)
        else:
            if 'diagnosis' in file['meta']['clinical'] and file['meta']['clinical']['diagnosis']:
                if file['meta']['clinical']['diagnosis'] == 'basal cell carcinoma':
                    y.append(1)
                else:
                    y.append(0)
            else:
                 continue

        img = imread(file_names[i+1])
        img_resize = resize(img, (25,25,3))
        new_file_name = file_names[i+1].replace('SampledImages', 'processed_images').replace('.jpeg','_small.jpeg')
        imsave(new_file_name, img_resize)

        X.append(img_resize)

    return X,y

### Setting up our training and testing sets

At this point you should have `X` and `y` loaded. I stored them, combined in a single file called `data_array.npy`. Compute the count of malignant and benign by counting and printing the frequency of the two classes. You will see that the benign class is much more frequent. As a result we are going to consider using sythetic minority oversampling to deal with the imbalenced class. You need to install the `imbalanced-learn` module conda install. We will use the *synthetic minority over-sampling* within that library for that task. Below is our train test split and resampling application. What you see below is our usual train test split, followed by a resampled train test split using SMOTE.



In [11]:
X_images = './X_data.npy'
y_images = './y_data.npy'
if 'data_array.npy' not in os.listdir('./'):
    X, y = images()
    np.save('./x_data.npy', X)
    np.save('./y_data.npy', y)
    #np.save('data_array.npy', np.hstack([x, y.reshape((-1,1))]))

In [12]:
data = np.load('./data_array.npy')
data.shape

(1766, 1876)

In [13]:
#X_images = './ISIC_images_X.npy'
#y_images = './ISIC_images_y.npy'

data_array_not_cached = not op.exists(X_images) and not op.exists(y_images)
print(data_array_not_cached)

False


In [14]:
if data_array_not_cached:
    X, y = images()

    np.save('ISIC_images_X.npy', X)
    np.save('ISIC_images_y.npy', y)
    np.save('data_array.npy', np.hstack([x, y.reshape((-1,1))]))
else:
    X = np.load(X_images)
    y = np.load(y_images)    

In [15]:
X.shape

(1766, 1875)

In [28]:
print("total count {}".format(y.shape[0]))
print("Number of malignant class {}".format(y.sum()))
print("Number of benign class {}".format(y.shape[0]))

total count 1766
Number of malignant class 239
Number of benign class 1766


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=42, ratio = 1.0)
X_resamp_train, y_resamp_train = sm.fit_sample(X_train, y_train)

Note: For reproducible results, througout we will include `random_state=42` as a parameter
to all possible funtions.

## Fitting and Evaluation

In this section we will first apply some basic classifiers to understand the speed and baseline performance results we should expect. We will start out with basic training test splits. Note that because we are not using crossvalidation at the start, we should expect that our testing results might be overly optimistic.

### 1. How well does the resampling work?

For our evaluation we will focus on 4 metrics. We note that we don't really have to worry about multiple classes since this is a binary classification problem. Thus we only look at the four metrics:

 * f1
 * accuracy
 * precision
 * recall
 
We look at this for training and test. You may find it convienient to create boilerplate code for this within a loop because we will be doing this kind of evalution many times. For our first experiment just look at

* k-Nearest Neighbors

In this case we will use k=3. Now compute the four metrics for the unresampled X_train, y_train and X_test, y_test.  For the first set of numbers you *might* see similar results to the following:

    Regular KNN N=3 neighbors
    accuracy_score on train  fit: 	     0.882
    f1_score  on train  fit: 	         0.61
    precision_score on train  fit: 	     0.884
    recall_score on train  fit: 	     0.466
    accuracy_score on test  fit: 	     0.805
    f1_score  on test  fit: 	         0.368
    precision_score on test  fit: 	     0.51
    recall_score on test  fit: 	         0.287

Consider recall and review the score. A 0.287 score means that for 100% of real malignant cases **only** 29% are flagged as being malignant. While a higher than desirable false alarms (lower precision score) might be tolorable because patents may be retested, low recall is not acceptable.  Next do this again, separately where training takes place on X_resamp_train, y_resamp_train but **test** on X_test, y_test and evaluate with the 4 metric. With this mixed training how does KNN with N=3 perform accross the training and testing?


In [80]:
# fill in
n_neighbors=3
knn = KNeighborsClassifier(n_neighbors)
knn.fit(X_train,y_train)
y_pred_train=knn.predict(X_train)
y_pred_test=knn.predict(X_test)
print("accuracy_score on train fit: {0:.3f}".format(accuracy_score(y_train, y_pred_train)))
print("F1_score on train fit: {0:.3f}".format(f1_score(y_train,y_pred_train)))
print("Precision_score on train fit: {0:.3f}".format(precision_score(y_train,y_pred_train)))
print("Recall_score on train fit: {0:.3f}".format(recall_score(y_train,y_pred_train)))

print("accuracy_score on test fit: {0:.3f}".format(accuracy_score(y_test, y_pred_test)))
print("f1_score on test fit: {0:.3f}".format(f1_score(y_test, y_pred_test)))
print("Precision_score on train fit: {0:.3f}".format(precision_score(y_test, y_pred_test)))
print("Recall_score on train fit: {0:.3f}".format(recall_score(y_test, y_pred_test)))


accuracy_score on train fit: 0.888
F1_score on train fit: 0.373
Precision_score on train fit: 0.854
Recall_score on train fit: 0.238
accuracy_score on test fit: 0.866
f1_score on test fit: 0.101
Precision_score on train fit: 0.333
Recall_score on train fit: 0.060


In [26]:
#resample
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X_resamp_train, y_resamp_train, test_size=0.3, random_state=42)

In [27]:
n_neighbors=3
knn = KNeighborsClassifier(n_neighbors)
knn.fit(X_train,y_train)
y_pred_train=knn.predict(X_train_rs)
y_pred_test=knn.predict(X_test_rs)
print("accuracy_score on train fit: {0:.3f}".format(accuracy_score(y_train, y_pred_train)))
print("F1_score on train fit: {0:.3f}".format(f1_score(y_train,y_pred_train)))
print("Precision_score on train fit: {0:.3f}".format(precision_score(y_train,y_pred_train)))
print("Recall_score on train fit: {0:.3f}".format(recall_score(y_train,y_pred_train)))

print("accuracy_score on test fit: {0:.3f}".format(accuracy_score(y_test, y_pred_test)))
print("f1_score on test fit: {0:.3f}".format(f1_score(y_test, y_pred_test)))
print("Precision_score on train fit: {0:.3f}".format(precision_score(y_test, y_pred_test)))
print("Recall_score on train fit: {0:.3f}".format(recall_score(y_test, y_pred_test)))


accuracy_score on train fit: 0.915
F1_score on train fit: 0.921
Precision_score on train fit: 0.854
Recall_score on train fit: 1.000
accuracy_score on test fit: 0.865
f1_score on test fit: 0.881
Precision_score on train fit: 0.797
Recall_score on train fit: 0.985


### 2. Basline with training on resampled data and testing with test data

We are now going to consider a sequence of algorithms and then ensemble versions of these algorithms. Because we will be using a number of algorithms and comparing their results you will want to write a loop. To strictly minimize copy past code and instead use template strings and set up your classifiers in an array so you can loop through. In this part you neet to use `cross_validate` from `sklearn.model_selection`. You are going to be calling this on the resampled data which has balanced classes. You should do 5-fold crossvalidation, you should make sure that it displays all the scoreing functions `scoring=['f1','accuracy','precision','recall']`, that it returns the training scores `return_train_score=True` and that you stratify using the `groups` optional argument. You are making your own reports here, where you print out the training and testing scores for each of the 4 metrics. Because `cross_validate` returns a score for each fold of the cross validataion, what you should show is the mean and the standard devation. We basically assume that the standard devation is the "error" in our estimate so for training accruacy you it might print `0.92 +/- 0.01` where the first number is the mean of the 5 accuracies for that classifier, and the second number is the stdev. Do this for these 4 classifiers:

1. Random Forest
2. k-Nearest Neighbors 
3. Extra Trees
4. Support Vector Machine



In [28]:
#Random Forest
rf = RandomForestClassifier(n_estimators=10)
print("Cross validation for RandomForestClassifier")
rf_cv = cross_validate(rf, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['train_accuracy'].mean(),rf_cv['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['train_f1'].mean(),rf_cv['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['train_precision'].mean(),rf_cv['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['train_recall'].mean(),rf_cv['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['test_accuracy'].mean(),rf_cv['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['test_f1'].mean(),rf_cv['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['test_precision'].mean(),rf_cv['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(rf_cv['test_recall'].mean(),rf_cv['test_recall'].std()))


Cross validation for RandomForestClassifier
acuracy_score on train fit: 1.00 +/- 0.00
f1_score on train fit: 1.00 +/- 0.00
precision_score on train fit: 1.00 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.91 +/- 0.01
f1_score on test fit: 0.91 +/- 0.01
precision_score on test fit: 0.89 +/- 0.01
recall_score on test fit: 0.92 +/- 0.02


In [29]:
#KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
print("Cross validation for KNeighborstClassifier")
knn_cv = cross_validate(kn, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['train_accuracy'].mean(),knn_cv['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['train_f1'].mean(),knn_cv['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['train_precision'].mean(),knn_cv['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['train_recall'].mean(),knn_cv['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['test_accuracy'].mean(),knn_cv['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['test_f1'].mean(),knn_cv['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['test_precision'].mean(),knn_cv['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(knn_cv['test_recall'].mean(),knn_cv['test_recall'].std()))


Cross validation for KNeighborstClassifier
acuracy_score on train fit: 0.92 +/- 0.00
f1_score on train fit: 0.93 +/- 0.00
precision_score on train fit: 0.87 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.88 +/- 0.01
f1_score on test fit: 0.89 +/- 0.01
precision_score on test fit: 0.81 +/- 0.01
recall_score on test fit: 1.00 +/- 0.00


In [30]:
#ExtraTree
tree = ExtraTreeClassifier()
print("Cross validation for ExtraClassifier")
tree_cv = cross_validate(tree, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['train_accuracy'].mean(),tree_cv['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['train_f1'].mean(),tree_cv['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['train_precision'].mean(),tree_cv['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['train_recall'].mean(),tree_cv['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['test_accuracy'].mean(),tree_cv['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['test_f1'].mean(),tree_cv['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['test_precision'].mean(),tree_cv['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(tree_cv['test_recall'].mean(),tree_cv['test_recall'].std()))


Cross validation for ExtraClassifier
acuracy_score on train fit: 1.00 +/- 0.00
f1_score on train fit: 1.00 +/- 0.00
precision_score on train fit: 1.00 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.83 +/- 0.01
f1_score on test fit: 0.84 +/- 0.01
precision_score on test fit: 0.79 +/- 0.01
recall_score on test fit: 0.88 +/- 0.01


In [31]:
#Support Vector Machine
svm = SVC(C=1.0, degree=3,gamma = 'auto')
print("Cross validation for Support Vector Machine")
svm_cv = cross_validate(svm, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['train_accuracy'].mean(),svm_cv['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['train_f1'].mean(),svm_cv['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['train_precision'].mean(),svm_cv['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['train_recall'].mean(),svm_cv['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['test_accuracy'].mean(),svm_cv['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['test_f1'].mean(),svm_cv['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['test_precision'].mean(),svm_cv['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(svm_cv['test_recall'].mean(),svm_cv['test_recall'].std()))


Cross validation for Support Vector Machine
acuracy_score on train fit: 0.67 +/- 0.01
f1_score on train fit: 0.66 +/- 0.01
precision_score on train fit: 0.69 +/- 0.01
recall_score on train fit: 0.63 +/- 0.01
acuracy score on test fit: 0.66 +/- 0.01
f1_score on test fit: 0.65 +/- 0.02
precision_score on test fit: 0.68 +/- 0.02
recall_score on test fit: 0.62 +/- 0.03


### 3. Bagging the classiefiers

So here we are going to see if bagging the classifiers can help. You are going to run the same 4 classifiers through but wrap them in a `BaggingClassifier` which will use boostrap resampling. You also will should start by trying 60% (.6) of the sampling data, and only 70% (.7) of the features to make each of the ensemble learners more independent. Start out with 20 estimators. Since you need to pass this config to each of the constructors you can make a dictionary of config variables and then use the python `**` to unpack it. So if the config variables are like this:

`bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)`

Then you use them like this:

`BaggingClassifier(KNeighborsClassifier(n_neighbors=3), **bagged_config)`

Write this as a loop over the classiffiers like above but for debuging purposes you can use the python `break` statement to only do one loop until you get it working. Each time through it should print the report like you made above shouing training and testing of each of the four matrics, using 5-fold cross_validataion which gives you a mean and a standard devation just like above. Compare the performance. Try adjusting the parameters such as max_samples or max_features to improve the performance.

Explain for each of the 4 bagged versions of your classifiers:

1. Random Forest
2. k-Nearest Neighbors 
3. Extra Trees
4. Support Vector Machine

explain why the results make sense or not.


In [35]:
#Bagging with  Randomforest 
bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)
bc = BaggingClassifier(base_estimator= RandomForestClassifier(n_estimators=10), **bagged_config)

print("Cross validation for Bagging Randomforest")
bc_rf = cross_validate(bc, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['train_accuracy'].mean(),bc_rf['train_accuracy'].std()))

print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['train_f1'].mean(),bc_rf['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['train_precision'].mean(),bc_rf['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['train_recall'].mean(),bc_rf['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['test_accuracy'].mean(),bc_rf['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['test_f1'].mean(),bc_rf['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['test_precision'].mean(),bc_rf['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_rf['test_recall'].mean(),bc_rf['test_recall'].std()))


Cross validation for Bagging Randomforest
acuracy_score on train fit: 0.98 +/- 0.00
f1_score on train fit: 0.99 +/- 0.00
precision_score on train fit: 0.98 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.89 +/- 0.01
f1_score on test fit: 0.90 +/- 0.01
precision_score on test fit: 0.85 +/- 0.02
recall_score on test fit: 0.95 +/- 0.01


In [37]:
#k-Nearest Neighbors
bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)
bc = BaggingClassifier(base_estimator= KNeighborsClassifier(n_neighbors=3), **bagged_config)

print("Cross validation for k-Nearest Neighbors")
bc_knn = cross_validate(bc, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_accuracy'].mean(),bc_knn['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_f1'].mean(),bc_knn['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_precision'].mean(),bc_knn['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_recall'].mean(),bc_knn['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_accuracy'].mean(),bc_knn['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_f1'].mean(),bc_knn['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_precision'].mean(),bc_knn['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_recall'].mean(),bc_knn['test_recall'].std()))


Cross validation for k-Nearest Neighbors
acuracy_score on train fit: 0.89 +/- 0.01
f1_score on train fit: 0.90 +/- 0.01
precision_score on train fit: 0.83 +/- 0.01
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.87 +/- 0.01
f1_score on test fit: 0.88 +/- 0.01
precision_score on test fit: 0.79 +/- 0.01
recall_score on test fit: 0.99 +/- 0.00


In [39]:
#Extra Trees
bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)
bc = BaggingClassifier(base_estimator= ExtraTreeClassifier(), **bagged_config)
print("bagging for k-Nearest Neighbors")

bc_knn = cross_validate(bc, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
              return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_accuracy'].mean(),bc_knn['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_f1'].mean(),bc_knn['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_precision'].mean(),bc_knn['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['train_recall'].mean(),bc_knn['train_recall'].std()))

print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_accuracy'].mean(),bc_knn['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_f1'].mean(),bc_knn['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_precision'].mean(),bc_knn['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_knn['test_recall'].mean(),bc_knn['test_recall'].std()))


bagging for k-Nearest Neighbors
acuracy_score on train fit: 1.00 +/- 0.00
f1_score on train fit: 1.00 +/- 0.00
precision_score on train fit: 1.00 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.91 +/- 0.02
f1_score on test fit: 0.91 +/- 0.01
precision_score on test fit: 0.87 +/- 0.02
recall_score on test fit: 0.96 +/- 0.00


In [42]:
#Support Vector Machine
bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)
svm = BaggingClassifier(base_estimator=SVC(C=1.0, degree=3,gamma = 'auto') , **bagged_config)

print("bagging for k-Nearest Neighbors")
bc_svm = cross_validate(svm, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
                                            return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['train_accuracy'].mean(),bc_svm['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['train_f1'].mean(),bc_svm['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['train_precision'].mean(),bc_svm['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['train_recall'].mean(),bc_svm['train_recall'].std()))


print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['test_accuracy'].mean(),bc_svm['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['test_f1'].mean(),bc_svm['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['test_precision'].mean(),bc_svm['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(bc_svm['test_recall'].mean(),bc_svm['test_recall'].std()))


bagging for k-Nearest Neighbors
acuracy_score on train fit: 0.66 +/- 0.01
f1_score on train fit: 0.65 +/- 0.01
precision_score on train fit: 0.68 +/- 0.01
recall_score on train fit: 0.63 +/- 0.02
acuracy score on test fit: 0.65 +/- 0.01
f1_score on test fit: 0.64 +/- 0.01
precision_score on test fit: 0.66 +/- 0.02
recall_score on test fit: 0.62 +/- 0.03


### 4. Adaptive Boosting

Here we will consider the adaptive boosting classifier. Not all learners can be boosted (unlike bagging). We will only consider two boosted algorithms here. 

1. Extra Tree
2. Descision Tree

In the case of decision tree use max_depth=2 and max_features .7. Here try 100 estimators in the `AdaBoostClassifier` wrapping the two algorothms. Just like above use 5 fold cross_validataion, random_seed=42 everywhere, and print out your reports to be able to compare. Play with the paramters a bit to try to get the best results possible. 


In [45]:
# fill in
tree = AdaBoostClassifier(base_estimator=ExtraTreeClassifier(),n_estimators=100,random_state=42)
print("Adaptive boosting for Extra tree")
Ad_tree = cross_validate(tree, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
                                            return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['train_accuracy'].mean(),Ad_tree['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['train_f1'].mean(),Ad_tree['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['train_precision'].mean(),Ad_tree['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['train_recall'].mean(),Ad_tree['train_recall'].std()))


print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['test_accuracy'].mean(),Ad_tree['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['test_f1'].mean(),Ad_tree['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['test_precision'].mean(),Ad_tree['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(Ad_tree['test_recall'].mean(),Ad_tree['test_recall'].std()))


Adaptive boosting for Extra tree
acuracy_score on train fit: 1.00 +/- 0.00
f1_score on train fit: 1.00 +/- 0.00
precision_score on train fit: 1.00 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.81 +/- 0.02
f1_score on test fit: 0.82 +/- 0.01
precision_score on test fit: 0.78 +/- 0.02
recall_score on test fit: 0.86 +/- 0.01


In [46]:
#Descision Tree
tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',max_depth=2, max_features = 0.7),
                                                                                n_estimators=100,random_state=42)
print("Adaptive boosting for Extra tree")
dc_tree = cross_validate(tree, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
                                            return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['train_accuracy'].mean(),dc_tree['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['train_f1'].mean(),dc_tree['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['train_precision'].mean(),dc_tree['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['train_recall'].mean(),dc_tree['train_recall'].std()))


print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['test_accuracy'].mean(),dc_tree['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['test_f1'].mean(),dc_tree['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['test_precision'].mean(),dc_tree['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['test_recall'].mean(),dc_tree['test_recall'].std()))


Adaptive boosting for Extra tree
acuracy_score on train fit: 1.00 +/- 0.00
f1_score on train fit: 1.00 +/- 0.00
precision_score on train fit: 1.00 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.91 +/- 0.02
f1_score on test fit: 0.92 +/- 0.02
precision_score on test fit: 0.88 +/- 0.03
recall_score on test fit: 0.97 +/- 0.03


### 5. Gradient Boosting

Below do exactly as above with Adaptive boosting but now with gradient boosting. Unfortunately you will not be able to use ExtraTrees here so you will end up just gradient boosting the default Decision Tree classifier. Again report the results and as before do 5 fold cross valitation. 


In [48]:
gb = GradientBoostingClassifier(n_estimators=100,max_depth=2, max_features = 0.7, random_state=42)
print("Gradient Boosting with Decision tree")
gb_tree = cross_validate(gb, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
                                            return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['train_accuracy'].mean(),gb_tree['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['train_f1'].mean(),gb_tree['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['train_precision'].mean(),gb_tree['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['train_recall'].mean(),gb_tree['train_recall'].std()))


print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['test_accuracy'].mean(),gb_tree['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(dc_tree['test_f1'].mean(),gb_tree['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['test_precision'].mean(),gb_tree['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(gb_tree['test_recall'].mean(),gb_tree['test_recall'].std()))



Gradient Boosting with Decision tree
acuracy_score on train fit: 0.94 +/- 0.00
f1_score on train fit: 0.95 +/- 0.00
precision_score on train fit: 0.91 +/- 0.01
recall_score on train fit: 0.98 +/- 0.00
acuracy score on test fit: 0.88 +/- 0.02
f1_score on test fit: 0.92 +/- 0.02
precision_score on test fit: 0.84 +/- 0.02
recall_score on test fit: 0.93 +/- 0.02


### Hard Voting classifier

Here you are going to implement a card voting classifier. You will pick three diverse learners of your choice. For a group of learners to be diverse means that they make different errors on the same data. In other words, one might say that the errors made by the classifers are uncorrelated.

Use sklearn's VotingClassifier to bind them together and again use cross_validate to score. 

In [49]:
rf = RandomForestClassifier(n_estimators=100)
knn = KNeighborsClassifier(n_neighbors=3)
tree=ExtraTreeClassifier()

hv = VotingClassifier(estimators=[('randomForest', rf), ('KNeigbors', knn), ('Extra_Tree', tree)], voting='hard')
print("Hard voting classifier")
hvc = cross_validate(hv, X_resamp_train, y_resamp_train, cv = 5, scoring=['f1','accuracy','precision','recall'], 
                                            return_train_score = True)
print("acuracy_score on train fit: {0:0.2f} +/- {1:0.2f}".format(hvc['train_accuracy'].mean(),hvc['train_accuracy'].std()))
print("f1_score on train fit: {0:0.2f} +/- {1:0.2f}".format(hvc['train_f1'].mean(),hvc['train_f1'].std()))
print("precision_score on train fit: {0:0.2f} +/- {1:0.2f}".format(hvc['train_precision'].mean(),hvc['train_precision'].std()))
print("recall_score on train fit: {0:0.2f} +/- {1:0.2f}".format(hvc['train_recall'].mean(),hvc['train_recall'].std()))


print("acuracy score on test fit: {0:0.2f} +/- {1:0.2f}".format(hvc['test_accuracy'].mean(),hvc['test_accuracy'].std()))
print("f1_score on test fit: {0:0.2f} +/- {1:0.2f}".format(hvc['test_f1'].mean(),hvc['test_f1'].std()))
print("precision_score on test fit: {0:0.2f} +/- {1:0.2f}".format(hvc['test_precision'].mean(),hvc['test_precision'].std()))
print("recall_score on test fit: {0:0.2f} +/- {1:0.2f}".format(hvc['test_recall'].mean(),hvc['test_recall'].std()))



Hard voting classifier
acuracy_score on train fit: 1.00 +/- 0.00
f1_score on train fit: 1.00 +/- 0.00
precision_score on train fit: 1.00 +/- 0.00
recall_score on train fit: 1.00 +/- 0.00
acuracy score on test fit: 0.92 +/- 0.01
f1_score on test fit: 0.92 +/- 0.01
precision_score on test fit: 0.87 +/- 0.01
recall_score on test fit: 0.98 +/- 0.00


## Conclusion

Write a conclusion explaining what we should take away from the results.

**Acknowlegement: Many Thanks to Toma Suciu for the first draft**

I think precision and recall are important factors to be consider because precision measure the classifiers exactness 
and recall measure the classifiers completeness. A low recall indicates many False Negatives.
precision = TP/(TP + FP)<br/>
recall = TP/(TP + FN)<br/>
Decision tree has performed well